In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

# Sample food order sequences
order_sequences = [
    ["Pizza", "Burger", "Sushi", "Pizza", "Salad", "Sushi", "Burger", "Pizza"],
    ["Burger", "Salad", "Pizza", "Burger", "Sushi", "Salad"],
    ["Pizza", "Sushi", "Burger", "Salad", "Pizza"]
]

# Step 1: Build a transition matrix
transition_counts = defaultdict(lambda: defaultdict(int))

# Count transitions
for sequence in order_sequences:
    for i in range(len(sequence) - 1):
        current_food = sequence[i]
        next_food = sequence[i + 1]
        transition_counts[current_food][next_food] += 1

# Create the transition probability matrix
food_items = list({food for seq in order_sequences for food in seq})
transition_matrix = pd.DataFrame(index=food_items, columns=food_items, data=0.0)

for current_food, next_foods in transition_counts.items():
    total_transitions = sum(next_foods.values())
    for next_food, count in next_foods.items():
        transition_matrix.loc[current_food, next_food] = count / total_transitions

# Fill NaN values with 0 for no transitions
transition_matrix.fillna(0, inplace=True)

print("Transition Matrix:")
print(transition_matrix)

# Step 2: Predict the next food category
def predict_next_order(current_food, transition_matrix):
    if current_food not in transition_matrix.index:
        print(f"No data available for {current_food}. Cannot make a prediction.")
        return None

    # Get the probabilities for the next food item
    next_food_probabilities = transition_matrix.loc[current_food]
    most_likely_food = next_food_probabilities.idxmax()

    return most_likely_food

# Example prediction
current_order = "Pizza"
predicted_next_order = predict_next_order(current_order, transition_matrix)
print(f"The predicted next order after {current_order} is: {predicted_next_order}")

# Step 3: Simulate a sequence of future orders
def simulate_orders(start_food, transition_matrix, n=5):
    current_food = start_food
    order_sequence = [current_food]

    for _ in range(n):
        next_food = predict_next_order(current_food, transition_matrix)
        if next_food is None:
            break
        order_sequence.append(next_food)
        current_food = next_food

    return order_sequence

# Example simulation
start_food = "Pizza"
simulated_orders = simulate_orders(start_food, transition_matrix, n=10)
print(f"Simulated sequence of future orders starting from {start_food}:")
print(simulated_orders)


Transition Matrix:
        Burger     Pizza     Sushi  Salad
Burger     0.0  0.200000  0.400000   0.40
Pizza      0.5  0.000000  0.250000   0.25
Sushi      0.5  0.250000  0.000000   0.25
Salad      0.0  0.666667  0.333333   0.00
The predicted next order after Pizza is: Burger
Simulated sequence of future orders starting from Pizza:
['Pizza', 'Burger', 'Sushi', 'Burger', 'Sushi', 'Burger', 'Sushi', 'Burger', 'Sushi', 'Burger', 'Sushi']
